In [0]:
table_dict={i.name.replace('/',''):i.path for i in dbutils.fs.ls('/mnt/silver/public')}
gold_writes={}

In [0]:
dataframes={}
for name,path in table_dict.items():
    dataframes[name]=spark.read.format('delta').load(path)

In [0]:
#Total Sales by Year and Month
from pyspark.sql.functions import month, year, sum,to_date, col
from pyspark.sql.types import DecimalType
decimal_type = DecimalType(10,2)
dataframes['Order']=dataframes['Order'].withColumn('order_date',to_date(col("order_date"),"MM/dd/yyyy"))
dataframes["Order"]=dataframes["Order"].withColumn('year',year(col("order_date")).alias("year")).withColumn('month',month(col("order_date")).alias("month"))
total_sales_df = dataframes['Order'].groupBy("year","month").agg(sum("total_price").cast(decimal_type).alias("total_sales"))
total_sales_df.show()
gold_writes['total_sales_df']='/mnt/gold/public/total_sales/'


+----+-----+-----------+
|year|month|total_sales|
+----+-----+-----------+
|2024|    7|50506040.83|
|2023|    9|48806898.85|
|2023|    8|38314491.17|
|2024|    3|51626344.90|
|2024|    5|51442699.17|
|2024|    2|48084953.53|
|2023|   11|50059485.56|
|2024|    1|49856095.00|
|2023|   10|50880037.02|
|2024|    6|50920734.01|
|2023|   12|49979364.12|
|2024|    4|49267669.47|
|2024|    8|10533589.54|
+----+-----+-----------+



In [0]:
#Top-Selling Car Models
from pyspark.sql.functions import count,col
order_df = dataframes['Order']
car_df = dataframes['Car']
top_selling_cars_df = order_df.alias("o") \
    .join(car_df.alias("c"), col("o.car_id") == col("c.Id")) \
    .groupBy("c.company", "c.model") \
    .agg(count("o.Id").alias("sales_count")) \
    .orderBy(col("sales_count").desc())
top_selling_cars_df.show()
gold_writes['top_selling_cars_df']='/mnt/gold/public/top_selling_cars/'

+----------+--------------------+-----------+
|   company|               model|sales_count|
+----------+--------------------+-----------+
|    Jaguar|                  XK|       1485|
|    Jaguar|                  XE|       1393|
|    Jaguar|                  XJ|       1291|
|Land Rover|         Range Rover|       1270|
|Land Rover|Range Rover Velar...|       1260|
|    Jaguar|              E-PACE|       1250|
|    Jaguar|              F-PACE|       1186|
|    Jaguar|              I-PACE|       1181|
|Land Rover|           Discovery|       1179|
|Land Rover|Range Rover Sport...|       1132|
|    Jaguar|              F-Type|       1108|
|    Jaguar|                  XF|       1082|
|Land Rover|   Range Rover Sport|       1080|
|Land Rover|   Range Rover Velar|       1062|
|Land Rover|            Defender|       1041|
|Land Rover|  Range Rover Evoque|       1011|
|Land Rover|     Discovery Sport|        989|
+----------+--------------------+-----------+



In [0]:
#sales by employee
from pyspark.sql.functions import sum
from pyspark.sql.types import DecimalType
decimal_type = DecimalType(10,2)
sales_transaction_df = dataframes['SalesTransaction']
employee_df = dataframes['Employee']
sales_by_employee_df = sales_transaction_df.join(employee_df, sales_transaction_df.employee_id == employee_df.Id).groupBy("first_name", "last_name").agg(sum("amount_paid").cast(decimal_type).alias("total_sales"))
sales_by_employee_df.show()
gold_writes['sales_by_employee_df']='/mnt/gold/public/sales_by_employee/'

+----------+----------+-----------+
|first_name| last_name|total_sales|
+----------+----------+-----------+
|  Benjamin|      Webb|   13337.86|
|   Michael|   Montoya|    7829.28|
|      Jose|Livingston|   11140.07|
|     Jesus|    Flores|   11512.42|
|     Aaron|  Saunders|    7214.58|
|      Dale|    Morgan|   13665.71|
|   Kristen|   Elliott|    9487.01|
|   William|  Mitchell|    7978.05|
|    Joshua|  Anderson|    8173.30|
|       Amy|    Barker|    5828.12|
|      Mark|      Khan|    7015.97|
|     Traci|  Cardenas|    8312.62|
|     Jorge|      Ware|    6541.79|
|     Brent|  Robinson|    7569.25|
|    Ashley|    Rogers|    8631.51|
|     Robyn|   Collins|    3140.50|
|     Craig|     Brown|    3730.55|
|   Sabrina|    Miller|    5878.63|
|      Tina|     Olsen|    9929.37|
|   Richard|   Summers|   12081.38|
+----------+----------+-----------+
only showing top 20 rows



In [0]:
# Customer Lifetime Value
from pyspark.sql.types import DecimalType
decimal_type = DecimalType(10,2)
order_df=dataframes['Order']
customer_df=dataframes['Customer']
customer_lifetime_value_df = order_df.join(customer_df, order_df.customer_id == customer_df.Id).groupBy("first_name", "last_name").agg(sum("total_price").cast(decimal_type).alias("lifetime_value"))
customer_lifetime_value_df.show()
gold_writes['customer_lifetime_value_df']='/mnt/gold/public/customer_lifetime_value/'

+----------+---------+--------------+
|first_name|last_name|lifetime_value|
+----------+---------+--------------+
|   Phillip|      Lee|      63371.23|
|     Emily|    Price|     138286.06|
|    Joseph|    Cowan|     104222.15|
|     Tonya| Thompson|      58196.79|
|      Mary|    Bowen|      39461.49|
|  Kimberly|   Martin|      18470.43|
|    George|  Johnson|      37880.29|
| Elizabeth|   Bailey|      46974.44|
|     Randy|    Adams|      14496.74|
|      Kara|     Ward|      44762.00|
|    Olivia|   Malone|      44160.54|
|    Pamela|  Higgins|      61334.14|
|     James|    Smith|     194073.40|
|  Mitchell|    Wells|      55837.03|
|   Natalie|   Austin|      50864.28|
|     Diana|   Sutton|      27721.82|
|    Nicole|    Stein|      10836.56|
|     Tanya|  Collins|      38905.21|
|   Michael|  Montoya|      46200.35|
|     Brett|    Brown|      30904.85|
+----------+---------+--------------+
only showing top 20 rows



In [0]:
# Most Serviced Cars
service_record_df=dataframes['ServiceRecord']
car_df=dataframes['Car']

most_serviced_cars_df = service_record_df \
.join(car_df, service_record_df.car_id == car_df.Id) \
.groupBy("company", "model") \
.agg(count("service_date").alias("service_count")) \
.orderBy(col("service_count").desc())
most_serviced_cars_df.show()
gold_writes['most_serviced_cars_df']='/mnt/gold/public/most_serviced_cars/'


+----------+--------------------+-------------+
|   company|               model|service_count|
+----------+--------------------+-------------+
|    Jaguar|                  XK|          872|
|    Jaguar|                  XE|          818|
|Land Rover|           Discovery|          790|
|    Jaguar|              I-PACE|          783|
|Land Rover|         Range Rover|          778|
|    Jaguar|                  XJ|          770|
|    Jaguar|                  XF|          738|
|    Jaguar|              F-PACE|          735|
|Land Rover|Range Rover Velar...|          733|
|    Jaguar|              E-PACE|          730|
|    Jaguar|              F-Type|          721|
|Land Rover|   Range Rover Velar|          709|
|Land Rover|Range Rover Sport...|          664|
|Land Rover|   Range Rover Sport|          652|
|Land Rover|            Defender|          649|
|Land Rover|     Discovery Sport|          600|
|Land Rover|  Range Rover Evoque|          593|
+----------+--------------------+-------

In [0]:
#Average Maintenance Cost and Frequency by Car Model and Year
from pyspark.sql.functions import col, year, sum, count, avg
from pyspark.sql.types import DecimalType
decimal_type = DecimalType(10,2)
maintenance_record_df = dataframes['MaintenanceRecord'].withColumn('maintenance_date',to_date(col("maintenance_date"),"MM/dd/yyyy"))

maintenance_record_df = maintenance_record_df.withColumn('maintenance_year',year(col("maintenance_date")))
car_df = dataframes['Car']

maintenance_cost_frequency_df = maintenance_record_df.alias("m") \
    .join(car_df.alias("c"), col("m.car_id") == col("c.Id")) \
    .groupBy("c.company", "c.model", "maintenance_year") \
    .agg(
        count("m.Id").alias("maintenance_count"),
        sum("m.cost").alias("total_maintenance_cost"),
        avg("m.cost").alias("avg_maintenance_cost_per_event")
    ) \
    .orderBy("c.company", "c.model", "maintenance_year")
maintenance_cost_frequency_df.show()
gold_writes['maintenance_cost_frequency_df']='/mnt/gold/public/maintenance_cost_frequency/'

+----------+---------+----------------+-----------------+----------------------+------------------------------+
|   company|    model|maintenance_year|maintenance_count|total_maintenance_cost|avg_maintenance_cost_per_event|
+----------+---------+----------------+-----------------+----------------------+------------------------------+
|    Jaguar|   E-PACE|            2023|              229|     68107.29999999999|            297.41179039301306|
|    Jaguar|   E-PACE|            2024|              354|     97898.61000000006|              276.549745762712|
|    Jaguar|   F-PACE|            2023|              248|     71613.97999999998|             288.7660483870967|
|    Jaguar|   F-PACE|            2024|              396|    110172.13999999996|            278.21247474747463|
|    Jaguar|   F-Type|            2023|              204|              54891.01|            269.07357843137254|
|    Jaguar|   F-Type|            2024|              331|     94002.15000000013|             283.9944108

In [0]:
#writing to gold layer
for df_name,path in gold_writes.items():
    df=globals()[df_name]
    df.write.mode("overwrite").format("delta").save(path)